In [ ]:
import requests
from concurrent.futures import ThreadPoolExecutor
import pandas as pd

def fetch_data(url):
    response = requests.get(url)
    if response.status_code == 200:
        return response.json()
    else:
        print(f"Failed to fetch data from {url}")
        return None

def fetch_umkm_data(base_url, page):
    page_url = f"{base_url}/{page}"
    result = fetch_data(page_url)
    if result and 'empData' in result:
        return result['empData']
    else:
        return []

def fetch_all_umkm_data(base_url, total_records):
    num_pages = total_records // 10 + 1  
    with ThreadPoolExecutor(max_workers=10) as executor:
        futures = [executor.submit(fetch_umkm_data, base_url, page) for page in range(1, num_pages + 1)]

        results = []
        for future in futures:
            results.extend(future.result())
            if len(results) >= total_records:
                break

    return results[:total_records]

def main():
    base_url = "https://data.diskopukm.jatimprov.go.id/satu_data/webnya/loadData/0"
    total_records = 150000

    umkm_data = fetch_all_umkm_data(base_url, total_records)
    
    if umkm_data:
        umkm_df = pd.DataFrame(umkm_data)
        print(umkm_df.to_string(index=False))

        print(f"{total_records} data.")

if __name__ == "__main__":
    main()